In [119]:
import json
import pandas as pd
import re
import nltk
import ast
import os
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Define and ensure NLTK data directory exists
nltk_data_dir = os.path.join(os.getcwd(), 'nltk_data')
if not os.path.exists(nltk_data_dir):
    os.makedirs(nltk_data_dir)

if nltk_data_dir not in nltk.data.path:
    nltk.data.path.append(nltk_data_dir)

# Download the main 'punkt' resource
print("Attempting to download NLTK 'punkt' resource...")
nltk.download('punkt', download_dir=nltk_data_dir)

print("Attempting to download NLTK 'punkt_tab' resource as suggested by error...")
try:
    nltk.download('punkt_tab', download_dir=nltk_data_dir)
    print("Successfully downloaded or found 'punkt_tab'.")
except ValueError as ve: # NLTK often raises ValueError for unknown resources
    print(f"Note: 'punkt_tab' may not be a direct downloadable item: {ve}. The main 'punkt' resource should suffice if complete.")
except Exception as e:
    print(f"Note: Error trying to download 'punkt_tab': {e}. The main 'punkt' resource should provide necessary components.")

# Verify nltk.data.path
print(f"NLTK data path: {nltk.data.path}")

Attempting to download NLTK 'punkt' resource...
Attempting to download NLTK 'punkt_tab' resource as suggested by error...
Successfully downloaded or found 'punkt_tab'.
NLTK data path: ['d:/CapstoneProjectCC/notebook/nltk_data', 'd:\\CapstoneProjectCC\\notebook\\nltk_data']


[nltk_data] Downloading package punkt to
[nltk_data]     d:\CapstoneProjectCC\notebook\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     d:\CapstoneProjectCC\notebook\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [120]:
file_path_resep = '../data/resep.csv'  # Sesuaikan path jika perlu
df = pd.DataFrame() # Inisialisasi DataFrame kosong
try:
    df = pd.read_csv(file_path_resep)
    print(f"File '{file_path_resep}' berhasil dimuat. Jumlah baris awal: {len(df)}")
except FileNotFoundError:
    print(f"Error: File '{file_path_resep}' tidak ditemukan. Pastikan path sudah benar.")
except Exception as e:
    print(f"Error saat memuat file CSV: {e}")

File '../data/resep.csv' berhasil dimuat. Jumlah baris awal: 174


In [121]:
print("\nPratinjau 5 baris pertama data awal:")
print(df.head())


Pratinjau 5 baris pertama data awal:
                                               judul  \
0        377. Marmer Cake Simpel (Metode All In One)   
1           1186. Mochi miniatur kelapa versi simple   
2  Burnt Kentang Yogurt Cheese | Diblender, Simpe...   
3                         Wedang Kunyit Madu Praktis   
4                 Siomay Dimsum 3 bahan paling mudah   

                                                foto         penulis    porsi  \
0  https://img-global.cpcdn.com/recipes/7dce6b559...  Shanty Bambang  1 Orang   
1  https://img-global.cpcdn.com/recipes/6c42f0549...      Naqiyyah 🍒  2 Orang   
2  https://img-global.cpcdn.com/recipes/a9e84b765...     Zahrotul An  1 Orang   
3  https://img-global.cpcdn.com/recipes/e26be4b73...    Avita Unaiya  3 Orang   
4  https://img-global.cpcdn.com/recipes/b5e1aea64...   Manja Sari Ta  2 Orang   

                                               bahan  \
0  [{'grup': 'Bahan Utama', 'jumlah': '', 'bahan'...   
1  [{'grup': 'Bahan Utama'

In [122]:
if 'judul' in df.columns:
    initial_rows = len(df)
    df.dropna(subset=['judul'], inplace=True) # Hapus baris dengan judul NaN
    df = df[df['judul'].apply(lambda x: isinstance(x, str) and x.strip().lower() not in ['', 'judul tidak ditemukan'])]
    print(f"\nSetelah filter judul: {len(df)} baris tersisa (dari {initial_rows}).")
else:
    print("Peringatan: Kolom 'judul' tidak ditemukan. Tidak dapat melakukan filter berdasarkan judul.")


Setelah filter judul: 174 baris tersisa (dari 174).


In [123]:
if df.empty:
    print("DataFrame menjadi kosong setelah filter judul. Tidak dapat melanjutkan.")
    exit()

In [124]:
def extract_ingredients_from_string(bahan_str):
    """Mengekstrak daftar nama bahan dari string yang berisi list of dict."""
    if not isinstance(bahan_str, str):
        return "" # Kembalikan string kosong jika input bukan string
    try:
        # Sanitasi untuk kasus umum seperti 'null' JSON ke 'None' Python
        bahan_str_sanitized = bahan_str.replace('null', 'None').replace('true', 'True').replace('false', 'False')
        bahan_list_data = ast.literal_eval(bahan_str_sanitized)
        
        extracted_names = []
        if isinstance(bahan_list_data, list):
            for item_dict in bahan_list_data:
                if isinstance(item_dict, dict) and 'bahan' in item_dict and isinstance(item_dict['bahan'], str):
                    if item_dict['bahan'].strip(): # Pastikan nama bahan tidak kosong
                        extracted_names.append(item_dict['bahan'].strip())
        return ' '.join(extracted_names) # Gabungkan semua nama bahan menjadi satu string
    except (ValueError, SyntaxError, TypeError):
        return "" # Jika parsing gagal atau format tidak sesuai

In [125]:
if 'bahan' in df.columns:
    print("\nMemulai ekstraksi teks bahan dari kolom 'bahan'...")
    tqdm.pandas(desc="Ekstraksi bahan_text")
    df['bahan_text'] = df['bahan'].progress_apply(extract_ingredients_from_string)
    
    # Filter baris yang tidak memiliki bahan_text setelah ekstraksi
    initial_rows = len(df)
    df = df[df['bahan_text'].str.strip() != '']
    print(f"Setelah ekstraksi dan filter bahan_text kosong: {len(df)} baris tersisa (dari {initial_rows}).")
    
    if not df.empty:
        print("\nContoh 'bahan_text' setelah ekstraksi:")
        print(df[['judul', 'bahan_text']].head())
    else:
        print("DataFrame menjadi kosong setelah ekstraksi bahan_text. Tidak dapat melanjutkan.")
        exit()
else:
    print("Peringatan: Kolom 'bahan' tidak ditemukan. Tidak dapat mengekstrak 'bahan_text'.")
    exit()


Memulai ekstraksi teks bahan dari kolom 'bahan'...


Ekstraksi bahan_text: 100%|██████████| 174/174 [00:00<00:00, 2896.87it/s]

Setelah ekstraksi dan filter bahan_text kosong: 172 baris tersisa (dari 174).

Contoh 'bahan_text' setelah ekstraksi:
                                               judul  \
0        377. Marmer Cake Simpel (Metode All In One)   
1           1186. Mochi miniatur kelapa versi simple   
2  Burnt Kentang Yogurt Cheese | Diblender, Simpe...   
3                         Wedang Kunyit Madu Praktis   
4                 Siomay Dimsum 3 bahan paling mudah   

                                          bahan_text  
0  pasta Coklat Coklat Bubuk Air Panas Bahan Bias...  
1  Daging Kelapa Santan Rose Brand Tapioka Rose B...  
2  Kentang Kukus Yogurt Plain Telur Mentega Cair ...  
3         Bubuk Kunyit Desaku Air Panas Madu Es Batu  
4  Dada Ayam Segar Wortel Es Batu Bawang Putih Ta...  


In [126]:
print("\nInisialisasi Stemmer Sastrawi...")
stemmer = StemmerFactory().create_stemmer()


Inisialisasi Stemmer Sastrawi...


In [127]:
slang_path = '../script/slang.txt'  # Sesuaikan path jika perlu
slang_dict = {}
try:
    with open(slang_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line or ' : ' not in line:
                continue
            slang, formal = line.split(' : ', 1) # Pemisah ' : '
            slang_dict[slang.strip()] = formal.strip()
    print(f"Kamus slang dari '{slang_path}' berhasil dimuat ({len(slang_dict)} entri).")
except FileNotFoundError:
    print(f"Peringatan: File kamus slang '{slang_path}' tidak ditemukan. Normalisasi slang mungkin tidak dilakukan.")
except Exception as e:
    print(f"Error saat memuat kamus slang: {e}")

Kamus slang dari '../script/slang.txt' berhasil dimuat (1953 entri).


In [128]:
stopword_path = '../script/stopwords-tag.txt'  # Sesuaikan path jika perlu
custom_stopwords = set()
try:
    with open(stopword_path, 'r', encoding='utf-8') as f:
        custom_stopwords = set(line.strip().lower() for line in f if line.strip())
    print(f"Daftar stopword dari '{stopword_path}' berhasil dimuat ({len(custom_stopwords)} entri).")
except FileNotFoundError:
    print(f"Peringatan: File stopword '{stopword_path}' tidak ditemukan. Penghapusan stopword standar mungkin tidak dilakukan.")
except Exception as e:
    print(f"Error saat memuat stopwords: {e}")

Daftar stopword dari '../script/stopwords-tag.txt' berhasil dimuat (994 entri).


In [129]:
def preprocess_text_bahan(text):
    """Membersihkan dan memproses teks bahan: lowercase, hapus angka & simbol, normalisasi slang, hapus stopwords, stemming."""
    if not isinstance(text, str) or not text.strip():
        return ""
    
    text = text.lower()  # Lowercasing
    text = re.sub(r'\d+', '', text)  # Hapus angka
    text = re.sub(r'[^a-z\s]', '', text)  # Hapus simbol, sisakan huruf dan spasi
    text = text.strip() # Hapus spasi berlebih di awal/akhir setelah regex

    tokens = word_tokenize(text) # Tokenisasi
    
    # Normalisasi slang
    normalized_tokens = [slang_dict.get(token, token) for token in tokens]
    
    # Hapus stopwords dan token pendek
    filtered_tokens = [token for token in normalized_tokens if token not in custom_stopwords and len(token) > 1]
    
    # Stemming
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    
    return ' '.join(stemmed_tokens)

In [130]:
if 'bahan_text' in df.columns:
    print("\nMemulai preprocessing teks bahan untuk 'clean_bahan'...")
    tqdm.pandas(desc="Preprocessing clean_bahan")
    df['clean_bahan'] = df['bahan_text'].progress_apply(preprocess_text_bahan)
    initial_rows = len(df)
    df.dropna(subset=['clean_bahan'], inplace=True) 
    df = df[df['clean_bahan'].str.strip() != ''] 
    print(f"Setelah preprocessing dan filter clean_bahan kosong: {len(df)} baris tersisa (dari {initial_rows}).")
    if not df.empty:
        print("\nContoh 'clean_bahan' setelah preprocessing:")
        print(df[['judul', 'clean_bahan']].head())
    else:
        print("DataFrame menjadi kosong setelah preprocessing clean_bahan. Tidak dapat melanjutkan ke TF-IDF.")
        exit()
else:
    print("Kolom 'bahan_text' tidak ditemukan, tidak dapat membuat 'clean_bahan'.")
    exit()


Memulai preprocessing teks bahan untuk 'clean_bahan'...


Preprocessing clean_bahan: 100%|██████████| 172/172 [00:33<00:00,  5.13it/s]

Setelah preprocessing dan filter clean_bahan kosong: 172 baris tersisa (dari 172).

Contoh 'clean_bahan' setelah preprocessing:
                                               judul  \
0        377. Marmer Cake Simpel (Metode All In One)   
1           1186. Mochi miniatur kelapa versi simple   
2  Burnt Kentang Yogurt Cheese | Diblender, Simpe...   
3                         Wedang Kunyit Madu Praktis   
4                 Siomay Dimsum 3 bahan paling mudah   

                                         clean_bahan  
0  pasta coklat coklat bubuk margarine gula telur...  
1  daging kelapa tapioka gambar tani maizena gula...  
2  kentang kukus yogurt plain telur mentega cair ...  
3                          bubuk kunyit madu es batu  
4  dada ayam wortel es batu bawang putih tapioka ...  


In [131]:
df.reset_index(drop=True, inplace=True)
print(f"\nDataFrame di-reset indexnya, jumlah baris saat ini untuk TF-IDF: {len(df)}")


DataFrame di-reset indexnya, jumlah baris saat ini untuk TF-IDF: 172


In [132]:
if not df.empty and 'clean_bahan' in df.columns:
    print("\nMemulai vektorisasi TF-IDF pada kolom 'clean_bahan'...")
    vectorizer = TfidfVectorizer() # Inisialisasi default
    
    try:
        tfidf_matrix = vectorizer.fit_transform(df['clean_bahan'])
        print(f"Matriks TF-IDF berhasil dibuat dengan dimensi: {tfidf_matrix.shape}")

        # Membuat DataFrame dari matriks TF-IDF
        tfidf_df = pd.DataFrame(
            tfidf_matrix.toarray(),
            columns=vectorizer.get_feature_names_out()
        )
        print("DataFrame TF-IDF berhasil dibuat.")

        # --- 9. Menggabungkan Judul dengan Fitur TF-IDF ---
        # Pastikan df['judul'] memiliki indeks yang sesuai dengan tfidf_df (sudah di-handle dengan reset_index sebelumnya)
        output_df_with_tfidf = pd.concat([df[['judul']], tfidf_df], axis=1)
        print("\nKolom 'judul' berhasil digabungkan dengan DataFrame TF-IDF.")
        print("Pratinjau data gabungan:")
        print(output_df_with_tfidf.head())

        # --- 10. Menyimpan Hasil TF-IDF ke CSV ---
        output_csv_filename_tfidf = '../data/resep_tfidf_features.csv' # Nama file output baru
        try:
            output_df_with_tfidf.to_csv(output_csv_filename_tfidf, index=False, encoding='utf-8')
            print(f"\nData TF-IDF (termasuk judul) berhasil disimpan ke '{output_csv_filename_tfidf}'")
        except Exception as e:
            print(f"Error saat menyimpan CSV TF-IDF: {e}")
            
    except ValueError as ve:
        print(f"ValueError selama TF-IDF: {ve}")
        print("Ini bisa terjadi jika 'clean_bahan' berisi semua entri kosong setelah preprocessing.")
    except Exception as e:
        print(f"Terjadi error saat proses TF-IDF: {e}")
else:
    print("Tidak ada data 'clean_bahan' yang valid untuk diproses TF-IDF.")

print("\nSkrip preprocessing selesai.")


Memulai vektorisasi TF-IDF pada kolom 'clean_bahan'...
Matriks TF-IDF berhasil dibuat dengan dimensi: (172, 348)
DataFrame TF-IDF berhasil dibuat.

Kolom 'judul' berhasil digabungkan dengan DataFrame TF-IDF.
Pratinjau data gabungan:
                                               judul  agaragar  almond  \
0        377. Marmer Cake Simpel (Metode All In One)       0.0     0.0   
1           1186. Mochi miniatur kelapa versi simple       0.0     0.0   
2  Burnt Kentang Yogurt Cheese | Diblender, Simpe...       0.0     0.0   
3                         Wedang Kunyit Madu Praktis       0.0     0.0   
4                 Siomay Dimsum 3 bahan paling mudah       0.0     0.0   

   alpukat  ambon  anak  asa  asam  asin  ayak  ...  vanila  vanili  vanilla  \
0      0.0    0.0   0.0  0.0   0.0   0.0   0.0  ...     0.0     0.0      0.0   
1      0.0    0.0   0.0  0.0   0.0   0.0   0.0  ...     0.0     0.0      0.0   
2      0.0    0.0   0.0  0.0   0.0   0.0   0.0  ...     0.0     0.0      0.0   
3


Data TF-IDF (termasuk judul) berhasil disimpan ke '../data/resep_tfidf_features.csv'

Skrip preprocessing selesai.
